In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from PIL import Image, ImageTk  # Importar Pillow para manejar JPEG

# Cargar el dataframe desde el CSV
df_secano = pd.read_csv("/mnt/c/Users/danie/OneDrive/Desktop/Proyectos/proyecto_final/Notebooks/df_secano_final.csv")

# Diccionario para traducir meses numéricos a nombres
meses = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo", 6: "Junio",
    7: "Julio", 8: "Agosto", 9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

# Crear la ventana principal
root = tk.Tk()
root.title("SecanoApp")
root.geometry("600x500")
root.configure(bg="#d8bfd8")  # Lila claro como fondo

# Cargar y verificar la imagen
image_path = "/mnt/c/Users/danie/OneDrive/Desktop/Proyectos/proyecto_final/Las-legumbres-madrileñas-0010395.jpeg"
try:
    print(f"Intentando cargar la imagen desde: {image_path}")
    imagen_pil = Image.open(image_path)
    print("Imagen cargada con éxito usando Pillow")
    imagen_pil = imagen_pil.resize((300, 200), Image.Resampling.LANCZOS)
    imagen = ImageTk.PhotoImage(imagen_pil)
    print("Imagen convertida a PhotoImage con éxito")
except Exception as e:
    print(f"Error al cargar o procesar la imagen: {e}")
    imagen = None

# Frame superior para la imagen
frame_imagen = tk.Frame(root, bg="#d8bfd8")
frame_imagen.pack(pady=10)

# Crear el label solo si la imagen se cargó correctamente
if imagen is not None:
    try:
        print("Creando el Label con la imagen")
        imagen_label = tk.Label(frame_imagen, image=imagen, bg="#d8bfd8")
        imagen_label.image = imagen  # Mantener la referencia en el label
        imagen_label.pack()
        print("Label creado y empaquetado con éxito")
    except Exception as e:
        print(f"Error al crear el Label: {e}")
        tk.Label(frame_imagen, text=f"Error al mostrar la imagen: {e}", bg="#d8bfd8", fg="red").pack()
else:
    tk.Label(frame_imagen, text="No se pudo cargar la imagen", bg="#d8bfd8", fg="red").pack()

# Variables para los menús desplegables
anio_var = tk.StringVar(root)
cultivo_var = tk.StringVar(root)
estacion_var = tk.StringVar(root)

# Frame para los controles iniciales
frame_inicial = tk.Frame(root, bg="#d8bfd8")
frame_inicial.pack(pady=20)

# Etiquetas y menús desplegables
tk.Label(frame_inicial, text="Selecciona el año:", bg="#d8bfd8", fg="#4b0082").pack(pady=5)
anio_menu = ttk.OptionMenu(frame_inicial, anio_var, "2025", "2025", "2026")
anio_menu.pack(pady=5)

tk.Label(frame_inicial, text="Selecciona el cultivo:", bg="#d8bfd8", fg="#4b0082").pack(pady=5)
cultivo_menu = ttk.OptionMenu(frame_inicial, cultivo_var, "cereales", "cereales", "legumbres")
cultivo_menu.pack(pady=5)

tk.Label(frame_inicial, text="Selecciona la estación:", bg="#d8bfd8", fg="#4b0082").pack(pady=5)
estaciones = df_secano["nombre"].unique().tolist()  # Lista de estaciones únicas del CSV
estacion_menu = ttk.OptionMenu(frame_inicial, estacion_var, estaciones[0], *estaciones)
estacion_menu.pack(pady=5)

# Botón para consultar
btn_consultar = tk.Button(frame_inicial, text="Consultar", bg="#9370db", fg="white", command=lambda: consultar())
btn_consultar.pack(pady=10)

# Área para mostrar resultados
resultado_frame = tk.Frame(root, bg="#d8bfd8")
resultado_frame.pack(pady=20, expand=True)  # Expand=True para centrar verticalmente

# Configurar el label con fuente más grande y centrado
resultado_texto = tk.Label(
    resultado_frame, 
    text="", 
    wraplength=550, 
    justify="center",  # Centrar el texto horizontalmente
    bg="#d8bfd8", 
    fg="#4b0082",
    font=("Arial", 10)  # Fuente más grande (tamaño 14, ajustable)
)
resultado_texto.pack(pady=20)

# Botón para volver (inicialmente oculto)
btn_volver = tk.Button(resultado_frame, text="Volver", bg="#9370db", fg="white", command=lambda: volver())
btn_volver.pack(pady=10)
btn_volver.pack_forget()  # Ocultar al inicio

# Función para consultar
def consultar():
    anio = int(anio_var.get())
    cultivo = cultivo_var.get()
    estacion = estacion_var.get()
    
    # Filtrar el dataframe
    df_filtrado = df_secano[(df_secano["anio"] == anio) & (df_secano["nombre"] == estacion)]
    aptos = df_filtrado[df_filtrado[cultivo] == 1]
    
    if aptos.empty:
        resultado_texto.config(text=f"Este año el cultivo de {cultivo} en {estacion} necesita riego asistido.")
    else:
        # Crear una lista de tuplas con (mes, semana, nombre_mes) para ordenar
        resultados = []
        for index, row in aptos.iterrows():
            mes_num = row["mes"]
            mes_nombre = meses[mes_num]
            semana = row["semana"]
            resultados.append((mes_num, semana, f"{mes_nombre} - Semana {semana}"))
        
        # Ordenar por mes y luego por semana
        resultados_ordenados = sorted(resultados, key=lambda x: (x[0], x[1]))
        
        # Extraer solo el texto ordenado
        resultados_texto = [item[2] for item in resultados_ordenados]
        
        # Eliminar duplicados manteniendo el orden
        resultados_unicos = []
        [resultados_unicos.append(x) for x in resultados_texto if x not in resultados_unicos]
        
        texto = f"Periodos aptos para {cultivo} en {estacion} ({anio}):\n"
        texto += "\n".join(resultados_unicos)
        resultado_texto.config(text=texto)
    
    # Ocultar controles iniciales y mostrar resultados
    frame_inicial.pack_forget()
    frame_imagen.pack_forget()  # Ocultar la imagen al mostrar resultados
    resultado_frame.pack(pady=20, expand=True)  # Centrar el frame
    btn_volver.pack()

# Función para volver atrás
def volver():
    # Limpiar resultados y mostrar controles iniciales
    resultado_texto.config(text="")
    resultado_frame.pack_forget()
    frame_imagen.pack(pady=10)  # Volver a mostrar la imagen
    frame_inicial.pack(pady=20)
    btn_volver.pack_forget()

# Iniciar la interfaz
root.mainloop()

Intentando cargar la imagen desde: /mnt/c/Users/danie/OneDrive/Desktop/Proyectos/proyecto_final/Las-legumbres-madrileñas-0010395.jpeg
Imagen cargada con éxito usando Pillow
Imagen convertida a PhotoImage con éxito
Creando el Label con la imagen
Label creado y empaquetado con éxito
